#### 04. Sentiment analysis

In [1]:
import pandas as pd
from transformers import pipeline
import numpy as np
from tqdm import tqdm
import pandas as pd
import plotly.express as px

import sys
sys.path.append("..")  # add project root to path

from src.sentiment_analysis import *

In [2]:
books = pd.read_csv("../data/books_with_categories.csv")

In [3]:
# Instantiates a Hugging Face text-classification pipeline using a pre-trained emotion detection model.
classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = "mps")

classifier("I love this!")


Device set to use mps


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.00852868054062128},
  {'label': 'neutral', 'score': 0.005764597561210394},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.0016119939973577857},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

As we can see, it classifies the sentence as 97.72% joy. Next, we will apply it to a book description.

In [4]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [5]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548424959182739},
  {'label': 'neutral', 'score': 0.169851616024971},
  {'label': 'sadness', 'score': 0.11640811711549759},
  {'label': 'surprise', 'score': 0.020700640976428986},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.015161189250648022},
  {'label': 'anger', 'score': 0.0039351521991193295}]]

According to the results, the emotion of `fear` is around 65.49%. However, when considering the entire book description, this impression does not really hold.

So, we are going to examine each sentence of the book description using the emotion classifier.

In [6]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.140385702252388},
  {'label': 'fear', 'score': 0.06816211342811584},
  {'label': 'joy', 'score': 0.04794247820973396},
  {'label': 'anger', 'score': 0.009156349115073681},
  {'label': 'disgust', 'score': 0.002628474263474345},
  {'label': 'sadness', 'score': 0.002122161677107215}],
 [{'label': 'neutral', 'score': 0.44937124848365784},
  {'label': 'disgust', 'score': 0.2735913395881653},
  {'label': 'joy', 'score': 0.1090826690196991},
  {'label': 'sadness', 'score': 0.09362722188234329},
  {'label': 'anger', 'score': 0.040478263050317764},
  {'label': 'surprise', 'score': 0.026970181614160538},
  {'label': 'fear', 'score': 0.006879068445414305}],
 [{'label': 'neutral', 'score': 0.6462150812149048},
  {'label': 'sadness', 'score': 0.24273425340652466},
  {'label': 'disgust', 'score': 0.0434226468205452},
  {'label': 'surprise', 'score': 0.02830052562057972},
  {'label': 'joy', 'score': 0.014211459085

So now we can see the feeling of each sentence. For example, the first sentence is `surprise`.

In [7]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [8]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [9]:
# Predictions for the first sentence of the description
predictions[0] 

[{'label': 'surprise', 'score': 0.7296027541160583},
 {'label': 'neutral', 'score': 0.140385702252388},
 {'label': 'fear', 'score': 0.06816211342811584},
 {'label': 'joy', 'score': 0.04794247820973396},
 {'label': 'anger', 'score': 0.009156349115073681},
 {'label': 'disgust', 'score': 0.002628474263474345},
 {'label': 'sadness', 'score': 0.002122161677107215}]

In [10]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

It is mostly classified as `surprised`.

In [11]:
# Predictions for the fourth sentence of the description
predictions[3] 

[{'label': 'fear', 'score': 0.9281691312789917},
 {'label': 'anger', 'score': 0.03219018131494522},
 {'label': 'neutral', 'score': 0.012808604165911674},
 {'label': 'sadness', 'score': 0.008756820112466812},
 {'label': 'surprise', 'score': 0.008597847074270248},
 {'label': 'disgust', 'score': 0.008431747555732727},
 {'label': 'joy', 'score': 0.0010455782758072019}]

However, we have introduced multiple emotions that can be associated with the same book. Therefore, for each book, we should have a separate column for each of the seven emotions. Then, for each particular emotion, we can take the highest probability across the entire description.

We see that the output for each sentence is ordered by score. Therefore, we first need to reorder the output to ensure the emotions appear in a consistent, predefined order.

In [12]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156349115073681},
 {'label': 'disgust', 'score': 0.002628474263474345},
 {'label': 'fear', 'score': 0.06816211342811584},
 {'label': 'joy', 'score': 0.04794247820973396},
 {'label': 'neutral', 'score': 0.140385702252388},
 {'label': 'sadness', 'score': 0.002122161677107215},
 {'label': 'surprise', 'score': 0.7296027541160583}]

We construct a function that calculates the maximum probability for each emotion across all sentences for a book’s description.

In [13]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "suprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels} # contains all the scores for each emotion for each description


def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels} # all the predictions for specific description
    for prediction in predictions: # sort them so that they have the emotions in the same order
        sorted_predictions = sorted(prediction, key=lambda x: x["label"]) 
        for index, label in enumerate(emotion_labels): # we extract the score for each emotion
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}  #take the maximum score 

Then, we process the first ten books by splitting each description into sentences, classifying the emotions of each sentence, extracting the maximum score for each emotion across the description, and storing these scores along with the corresponding ISBN. 

In [14]:
for i in range(10): # test on the first ten books
    isbn.append(books["isbn13"][i]) # add isbn13 value to isbn list
    sentences = books["description"][i].split(".") # split the description into sentences
    predictions = classifier(sentences) # estimate the predictions
    max_scores = calculate_max_emotion_scores(predictions) # get the maximum score
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label]) 

In [15]:
emotion_scores

{'anger': [np.float64(0.06413356214761734),
  np.float64(0.6126182675361633),
  np.float64(0.06413356214761734),
  np.float64(0.3514830470085144),
  np.float64(0.0814121812582016),
  np.float64(0.23222540318965912),
  np.float64(0.5381842255592346),
  np.float64(0.06413356214761734),
  np.float64(0.30067041516304016),
  np.float64(0.06413356214761734)],
 'disgust': [np.float64(0.2735913395881653),
  np.float64(0.3482859134674072),
  np.float64(0.10400660336017609),
  np.float64(0.15072262287139893),
  np.float64(0.1844949871301651),
  np.float64(0.7271743416786194),
  np.float64(0.15585485100746155),
  np.float64(0.10400660336017609),
  np.float64(0.279480904340744),
  np.float64(0.17792661488056183)],
 'fear': [np.float64(0.9281691312789917),
  np.float64(0.9425276517868042),
  np.float64(0.9723208546638489),
  np.float64(0.3607071042060852),
  np.float64(0.09504340589046478),
  np.float64(0.05136276036500931),
  np.float64(0.747427761554718),
  np.float64(0.40449652075767517),
  np.f

Now we apply `calculate_max_emotion_scores` to the entire dataset to extract the maximum emotion scores for each book description.

In [16]:
#reinstantiate the list here
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "suprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels} # contains all the scores for each emotion for each description


for i in tqdm(range(len(books))): # test on the first ten books
    isbn.append(books["isbn13"][i]) # add isbn13 value to isbn list
    sentences = books["description"][i].split(".") # split the description into sentences
    predictions = classifier(sentences) # estimate the predictions
    max_scores = calculate_max_emotion_scores(predictions) # get the maximum score
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label]) 

100%|██████████| 5197/5197 [41:17<00:00,  2.10it/s]   


In [17]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [18]:
emotions_df

,anger,disgust,fear,joy,sadness,suprise,neutral,isbn13
0,0.064134,0.273591,0.928169,0.932798,0.646215,0.967158,0.729603,9780002005883
1,0.612618,0.348286,0.942528,0.704422,0.887940,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150723,0.360707,0.251881,0.732687,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148209,0.030643,0.919165,0.255170,0.853722,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339216,0.947779,0.375756,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459271,0.759454,0.951104,0.368110,0.078765,9788185300535


In [19]:
books = pd.merge(books, emotions_df, on = "isbn13")

In [20]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,suprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928169,0.932798,0.646215,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348286,0.942528,0.704422,0.887940,0.111690,0.252546
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150723,0.360707,0.251881,0.732687,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148209,0.030643,0.919165,0.255170,0.853722,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339216,0.947779,0.375756,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459271,0.759454,0.951104,0.368110,0.078765


Then we take a look at the distribution of these emotions. We observe a good spread of emotions, with sadness showing relatively high probabilities. Overall, there is a good range, even though some influence appears in the lower values.

In [21]:

plot_emotion_distribution(emotions_df)

In [22]:
books.to_csv("../data/books_with_emotions.csv", index = False)